In [1]:
import numpy as np
import pandas as pd
import os
import math
os.getcwd()
os.chdir('C:\\Users\\EeLian\\Desktop\\OULAD_KAGGLE\\PythonCodeCleanUp\\Data')

### Dashboard_Table1_StudentInfo

In [2]:
# read datafile
studentInfo = pd.read_csv('studentInfo.csv')
courses = pd.read_csv('courses.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

# add _fl_latest_module_attended
studentInfo
studentInfo.sort_values(by= ['id_student','code_module','code_presentation'], inplace = True, ascending = False)
studentInfo['_fl_latest_module_attended'] = studentInfo.groupby(['id_student','code_module'])['id_student'].rank(method='first', ascending = False)
# studentInfo[studentInfo['id_student'].isin([584077])]

# merge/ left join to get studentInfo + courses + registration 
# name as studentInfo3 cos 3 files
studentInfo2 = studentInfo.merge(courses, how='left', left_on=["code_module", "code_presentation"], right_on=["code_module", "code_presentation"])
studentInfo3 = studentInfo2.merge(studentRegistration, how='left', left_on=["code_module", "code_presentation","id_student"], right_on=["code_module", "code_presentation","id_student"])
#studentInfo3['_flag_actual_dropout']=studentInfo3['date_unregistration'].isnull().astype(int)
studentInfo3.loc[studentInfo3['date_unregistration'] >0, '_flag_actual_dropout'] = 1
studentInfo3.loc[studentInfo3['date_unregistration'] <=0, '_flag_actual_dropout'] = 0
studentInfo3.loc[studentInfo3['date_unregistration'].isnull(), '_flag_actual_dropout'] = 0

studentInfo3.loc[((studentInfo3['_flag_actual_dropout']==0)&(studentInfo3['final_result'].isin(['Withdrawn']))), '_flag_prestart_withdrawal'] = 1
studentInfo3.loc[~((studentInfo3['_flag_actual_dropout']==0)&(studentInfo3['final_result'].isin(['Withdrawn']))), '_flag_prestart_withdrawal'] = 0

studentInfo3['_flag_final_result'] = pd.Series([], dtype=int)
studentInfo3.loc[(studentInfo3['final_result'].isin(['Withdrawn']))|(studentInfo3['final_result'].isin(['Fail'])), '_flag_final_result'] = 1
studentInfo3.loc[(studentInfo3['final_result'].isin(['Distinction']))|(studentInfo3['final_result'].isin(['Pass'])), '_flag_final_result'] = 0

studentInfo3=studentInfo3.replace({
    'imd_band': {
        '10-20': '10-20%'
    }
})
#EEE_Week16['imd_band']
#EEE_Week16['imd_band'].describe(include='all')


studentInfo3['_surrogate_key_one']=\
    studentInfo3['id_student'].astype(str) + \
    studentInfo3['final_result']+ \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_two']=\
    studentInfo3['final_result']+ \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_three']=\
    studentInfo3['id_student'].astype(str) + \
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']
studentInfo3['_surrogate_key_four']=\
    studentInfo3['code_module']+ \
    studentInfo3['code_presentation']


#studentInfo2
#studentInfo3

# export to csv
# studentInfo2.to_csv('studentInfo2.csv',index=True,header=True)
# studentInfo3.to_csv('DatasetForModel_StudentInfo.csv',index=True,header=True)

### Dashboard_Table2_StudentAssessment

In [3]:
studentAssessment = pd.read_csv('studentAssessment.csv')
assessments = pd.read_csv('assessments.csv')
# merge/ left join
# name as studentAssessment2
studentAssessment2 = studentAssessment.merge(assessments, how='left', left_on=["id_assessment"], right_on=["id_assessment"])

# add _flag_null_score just to check if there is submission
studentAssessment2['_flag_null_score']=studentAssessment2['score'].isnull().astype(int)
# add weighted score
studentAssessment2['_weighted_score'] = studentAssessment2['score']*studentAssessment2['weight']
# add pass fail. the NaN means didnt submit
studentAssessment2.loc[studentAssessment2['score'] >=40, '_flag_assessment_passed'] = 1
studentAssessment2.loc[studentAssessment2['score'] <40, '_flag_assessment_passed'] = 0
studentAssessment2["_date_group"] = pd.Series([], dtype=float)
studentAssessment2["_date_group"] = studentAssessment2['date_submitted']
studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_date_group'] = studentAssessment2['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

studentAssessment2.loc[studentAssessment2['date_submitted'] <0, '_precourse'] = 1
studentAssessment2.loc[studentAssessment2['date_submitted'] >=0, '_precourse'] = 0

studentInfo3_results_distinct=studentInfo3[['id_student','code_module','code_presentation','final_result']]
studentInfo3_results_distinct=studentInfo3_results_distinct.drop_duplicates()

studentAssessment3 = studentAssessment2.merge(studentInfo3_results_distinct, how='left', \
                                                 left_on=["id_student","code_module", "code_presentation"], \
                                                 right_on=["id_student","code_module", "code_presentation"])


studentAssessment3['_surrogate_key_one']=\
    studentAssessment3['id_student'].astype(str) + \
    studentAssessment3['final_result']+ \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_two']=\
    studentAssessment3['final_result']+ \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_three']=\
    studentAssessment3['id_student'].astype(str) + \
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']
studentAssessment3['_surrogate_key_four']=\
    studentAssessment3['code_module']+ \
    studentAssessment3['code_presentation']

studentAssessment3.loc[studentAssessment3['code_module'].isin(['AAA']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

studentAssessment3.loc[studentAssessment3['code_module'].isin(['BBB']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

studentAssessment3.loc[studentAssessment3['code_module'].isin(['CCC']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/200

studentAssessment3.loc[studentAssessment3['code_module'].isin(['DDD']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/200

studentAssessment3.loc[studentAssessment3['code_module'].isin(['EEE']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

studentAssessment3.loc[studentAssessment3['code_module'].isin(['FFF']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

studentAssessment3.loc[studentAssessment3['code_module'].isin(['GGG']), '_final_weighted_score'] = \
studentAssessment3['_weighted_score']/100

### Dashboard_Table3_StudentVle

In [4]:
studentVle = pd.read_csv('studentVle.csv')
studentVle2 = studentVle
vle = pd.read_csv('vle.csv')
studentVle3 = studentVle2.merge(vle, how='left', \
                                                 left_on=["code_module", "code_presentation","id_site"], \
                                                 right_on=["code_module", "code_presentation","id_site"])
#studentVle3.to_csv('studentVle3.csv',index=True,header=True)
studentVle3['_date_group'] = pd.Series([], dtype=float)
studentVle3['_date_group'] = studentVle3['date']
studentVle3.loc[studentVle3['date'] <0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.ceil(float(x/7)))
studentVle3.loc[studentVle3['date'] >=0, '_date_group'] = studentVle3['_date_group'].apply(lambda x: math.floor(float((x/7)+1)))

studentVle3.loc[studentVle3['date'] <0, '_precourse'] = 1
studentVle3.loc[studentVle3['date'] >=0, '_precourse'] = 0

studentInfo3_results_distinct=studentInfo3[['id_student','code_module','code_presentation','final_result']]
studentInfo3_results_distinct=studentInfo3_results_distinct.drop_duplicates()

studentVle4 = studentVle3.merge(studentInfo3_results_distinct, how='left', \
                                                 left_on=["id_student","code_module", "code_presentation"], \
                                                 right_on=["id_student","code_module", "code_presentation"])


studentVle4['_surrogate_key_one']=\
    studentVle4['id_student'].astype(str) + \
    studentVle4['final_result']+ \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_two']=\
    studentVle4['final_result']+ \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_three']=\
    studentVle4['id_student'].astype(str) + \
    studentVle4['code_module']+ \
    studentVle4['code_presentation']
studentVle4['_surrogate_key_four']=\
    studentVle4['code_module']+ \
    studentVle4['code_presentation']

studentInfo3
* keep: _surrogate_key_three, id_student, code_module, code_presentation 
* drop: _surrogate_key_one,_surrogate_key_two,_surrogate_key_four
* keep: _surrogate_key_two,_surrogate_key_one
    
studentAssessment3
* keep: _surrogate_key_three
* drop: _surrogate_key_one,_surrogate_key_two,_surrogate_key_four, id_student, code_module, code_presentation
    
studentVle4 
* keep: _surrogate_key_three
* drop: _surrogate_key_one,_surrogate_key_two,_surrogate_key_four, id_student, code_module, code_presentation

In [5]:
studentAssessment3.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,...,_weighted_score,_flag_assessment_passed,_date_group,_precourse,final_result,_surrogate_key_one,_surrogate_key_two,_surrogate_key_three,_surrogate_key_four,_final_weighted_score
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,...,780.0,1.0,3,0.0,Pass,11391PassAAA2013J,PassAAA2013J,11391AAA2013J,AAA2013J,7.8
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,...,700.0,1.0,4,0.0,Pass,28400PassAAA2013J,PassAAA2013J,28400AAA2013J,AAA2013J,7.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,...,720.0,1.0,3,0.0,Pass,31604PassAAA2013J,PassAAA2013J,31604AAA2013J,AAA2013J,7.2
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,...,690.0,1.0,4,0.0,Pass,32885PassAAA2013J,PassAAA2013J,32885AAA2013J,AAA2013J,6.9
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,...,790.0,1.0,3,0.0,Pass,38053PassAAA2013J,PassAAA2013J,38053AAA2013J,AAA2013J,7.9


In [6]:
List1_columns_drop=\
[
'_surrogate_key_one','_surrogate_key_two','_surrogate_key_four',\
    'id_student', 'code_module', 'code_presentation','final_result'
]
List2_columns_drop=\
[
'_surrogate_key_four'
]
studentInfo4=studentInfo3.drop(List2_columns_drop, axis = 1)
studentAssessment4=studentAssessment3.drop(List1_columns_drop, axis = 1)
studentInfo4.to_csv('Dataset_Dashboard_Table1_StudentInfo.csv',index=True,header=True)
studentAssessment4.to_csv('Dataset_Dashboard_Table1_StudentAssessment.csv',index=True,header=True)

In [7]:
studentVle5=studentVle4.drop(List1_columns_drop, axis = 1)
studentVle5.to_csv('Dataset_Dashboard_Table1_StudentVle.csv',index=True,header=True)

In [8]:
studentAssessment3.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,...,_weighted_score,_flag_assessment_passed,_date_group,_precourse,final_result,_surrogate_key_one,_surrogate_key_two,_surrogate_key_three,_surrogate_key_four,_final_weighted_score
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,...,780.0,1.0,3,0.0,Pass,11391PassAAA2013J,PassAAA2013J,11391AAA2013J,AAA2013J,7.8
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,...,700.0,1.0,4,0.0,Pass,28400PassAAA2013J,PassAAA2013J,28400AAA2013J,AAA2013J,7.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,...,720.0,1.0,3,0.0,Pass,31604PassAAA2013J,PassAAA2013J,31604AAA2013J,AAA2013J,7.2
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,...,690.0,1.0,4,0.0,Pass,32885PassAAA2013J,PassAAA2013J,32885AAA2013J,AAA2013J,6.9
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,...,790.0,1.0,3,0.0,Pass,38053PassAAA2013J,PassAAA2013J,38053AAA2013J,AAA2013J,7.9


In [9]:
studentAssessment4=studentAssessment3[studentAssessment3['is_banked'].isin([1])]
studentAssessment3_isbanked=studentAssessment4[['_surrogate_key_three','is_banked']]
studentAssessment3_isbanked=studentAssessment3_isbanked.drop_duplicates()
studentAssessment3_isbanked["_flag_isbanked"] = studentAssessment3_isbanked['is_banked']
studentAssessment3_isbanked=studentAssessment3_isbanked.drop(['is_banked'], axis = 1)
studentAssessment3_isbanked.to_csv('Dataset_Dashboard_Table1_StudentInfoIsBanked.csv',index=True,header=True)